### 파일 복붙 -> 제거
#### db에 있는 사용자인지 확인해서 모델 돌리고 결과값 나오게끔 반복문/조건문 만들기

In [107]:
# 필요 패키지 전부 다
from tensorflow.keras.models import load_model
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation
import tensorflow as tf
import numpy as np
import os
from numpy import argmax
import matplotlib.pyplot as plt             
import cv2                                 
from tqdm import tqdm
import shutil
from pathlib import Path
import json
from collections import OrderedDict

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# cpu만 강제 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# json 저장
after_table = OrderedDict()

In [108]:
'''
# 사용자 정보를 알 수 있는 게 필요함!!
# 새로운 이미지가 들어올 때 txt로 user 정보가 같이 들어오던가
# 새로운 이미지 파일명 자체가 '날짜_이름_전화번호' 의 형식으로 들어와야 누군지 확인 가능
'''

img_test = '2022-05-25-홍길동.jpg'  # 이미지 이름이 이렇다는 가정 하에
name = img_test.split('-',3)[3]
name = name.split('.')[0]
phone_number = '01012345678'

# 사진 위치 경로 / 복제 시킬 경로
base_path = './hair_image/before_hair/'
storage_path = './hair_image/storage/'

In [109]:
# 얼굴형 분류 모델 돌리기

def face_classify(name,phone_number,base_path,storage_path):
    for (root, directories, files) in os.walk(base_path):
        for file in files:
            file_path = os.path.join(root, file)
            fp = file_path.split('/',4)[3].split('_',1)[0]

            if fp == name:
                # storage 폴더에 같은 '이름_전화번호' 형식으로 폴더 생성 및 파일 복제
                storage = storage_path + name + '_' + phone_number
                # os.mkdir(storage)
                # shutil.copy(file_path, storage) 

                model = load_model('model.h5') # 모델 로드

                # 클래스 및 이미지 사이즈 지정하는 부분
                class_names = ['Heart', 'Oblong', 'Oval', 'Round', 'Square']
                class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

                nb_classes = len(class_names)

                IMAGE_SIZE = (150, 150)

                # 이미지 reshape
                # 파일 경로에 한글 있으므로 -> array -> decode -> cvt
                img_array = np.fromfile(file_path, np.uint8)
                test_image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
                test_image = cv2.resize(test_image, IMAGE_SIZE) 
                test_image = np.array(test_image, dtype = 'float32')
                test_image = test_image / 255.0
                test_image = test_image.reshape((-1,150,150,3))

                # 얼굴형 예측
                predictions = model.predict(test_image)
                pred_labels = np.argmax(predictions, axis = 1) 

                # 예측 label값 출력 --> face_shape 라는 이름으로 
                face_shape = class_names[int(pred_labels)]

                # 다 돌아갔으면 파일 지우기 --> 실행 X
                # Path(file_path).unlink()

                return face_shape

In [110]:
face_classify(name,phone_number,base_path,storage_path)

1/1 [==============================] - 0s 72ms/step


'Oblong'

In [111]:
# 사용자 정보 json 파일로 반환

after_table['name'] = name
after_table['face'] = face_shape
after_table['recom_hair'] = {"1":"보브" ,"2":"히피","3":"숏단발","4":"레이어드" }
after_table['before_hair'] = "원랭스"
after_table['after_hair'] = "보브"
after_table['satisfaction'] = "만족"

# with open('/content/drive/MyDrive/Colab Notebooks/user.json', 'w', encoding='utf-8') as json_file:
#   json.dump(after_table, json_file, ensure_ascii=False, indent='\t')

with open('./data/test.json', 'w', encoding='utf-8') as json_file:
  json.dump(after_table, json_file, ensure_ascii=False, indent='\t')

In [112]:
after_table

OrderedDict([('name', '홍길동'),
             ('face', 'Oblong'),
             ('recom_hair', {'1': '보브', '2': '히피', '3': '숏단발', '4': '레이어드'}),
             ('before_hair', '원랭스'),
             ('after_hair', '보브'),
             ('satisfaction', '만족')])